<a href="https://colab.research.google.com/github/Trailblazer29/Pain-Detection-From-Facial-Expressions/blob/main/Pain_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Pain Detection Through Facial Expressions - Ilham Seladji, MS. Data Analytics</center>

In [ ]:
#Load Libraries
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tkinter as tk
import re
from tkinter import filedialog
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import RegscorePy as rp

In [ ]:
# Load the OpenCV face cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
kernel = np.ones((5,5),np.float32)/25
def preprocess_img(img, img_size):
    #Face Detection & Cropping
    faces = face_cascade.detectMultiScale(img, 1.1, 4)
    # Grayscale conversion
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Histogram equalization
    img = cv2.equalizeHist(img)
    for (x, y, w, h) in faces: 
        img_ = img[y:y+h, x:x+w]
    try:
        #Resizing/Normalizing
        img_ = cv2.resize(img_,(img_size, img_size))
        #Mean Filtering
        img_ = cv2.filter2D(img_,-1,kernel)
    except Exception as e:
        #print("Exception: "+str(e))
        return img
    return img_

In [ ]:
def preprocess(class_path, img_size):
    for filename in os.listdir(class_path):
        #img = cv2.imread(os.path.join(class_path,filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.imread(os.path.join(class_path,filename))
        img = preprocess_img(img, img_size)
        cv2.imwrite(os.path.join("Processed "+class_path,filename), img)

# Preprocessing:

In [ ]:
preprocess("Binary Classified Pain Images/0", 224)
preprocess("Binary Classified Pain Images/1", 224)

## Print Sample

In [ ]:
for filename in os.listdir("Processed Binary Classified Pain Images/0"):
    #Conversion to grayscale
    img = cv2.imread(os.path.join("Processed Binary Classified Pain Images/0",filename), cv2.IMREAD_GRAYSCALE)
    tran = transforms.ToTensor()
    img = tran(img)
    break

# Importing Data

In [ ]:
data_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,)),
     transforms.Grayscale(num_output_channels=1)])
trainset = datasets.ImageFolder(root="train", transform=data_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 10, shuffle = True)

In [ ]:
testset = datasets.ImageFolder(root="test", transform=data_transform)
testloader = torch.utils.data.DataLoader(testset)

# Processing:

### CNN Architecture:

In [ ]:
num_epochs = 20
batch_size = 10
learning_rate = 0.0001
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
        nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 512, 1000)
        self.fc2 = nn.Linear(1000, 2)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out

### Computing Loss:

In [ ]:
model = NN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network:

In [ ]:
total_step = len(trainloader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),(correct / total) * 100))
        loss_list.append(loss.item())
        acc_list.append(correct / total)

In [ ]:
pd.DataFrame(acc_list).to_excel('accuracies.xlsx')
pd.DataFrame(loss_list).to_excel('losses.xlsx')

### Save Trained Model

In [ ]:
torch.save(model.state_dict(), "./pain_detection_nn.pth")

### Test Network Performance

In [ ]:
#load Model
model = NN()
model.load_state_dict(torch.load("pain_detection_nn.pth"))

<All keys matched successfully>

In [ ]:
model.eval()
actual_outputs=[]
predicted_outputs=[]
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        actual_outputs.append(labels.item())
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted_outputs.append(predicted.item())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = (correct / total) * 100
print('The test accuracy is: {} %'.format(accuracy))

The test accuracy is: 92.25 %


In [ ]:
cm = confusion_matrix(actual_outputs, predicted_outputs)
df_cm = pd.DataFrame(cm, index = ["No Pain", "Pain"],
                  columns = ["No Pain", "Pain"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

### Test Network with new images

In [ ]:
#Open Image
root = tk.Tk()
root.withdraw()
path = filedialog.askopenfilename()
new_img = cv2.imread(path)
processed_img = preprocess_img(new_img, 224)
plt.imshow(cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB))

In [ ]:
#Preprocess Image
tran = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])
processed_img = tran(processed_img)

In [ ]:
#Assess Pain
output = model(processed_img[None,...])

In [ ]:
value, index = torch.max(output, 1)
if index==0:
    print("No Pain")
else:
    print("Pain")
